In [50]:
from bs4 import BeautifulSoup as bs
import requests
import re

from pymongo import MongoClient
import requests
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs

from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import regex as re

import random

## Save HTML Search Results From Used Car Website (autolist.com)

In [153]:
path = "/Users/avantikagoyal/Desktop/DDR/chromedriver-mac-arm64/chromedriver"
driver = webdriver.Chrome(executable_path=path)

# Set wait and timeout parameters
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(900)

# Visit autolist.com
driver.get('https://www.autolist.com/')

# Click on the Price search button
time.sleep(5)  
driver.find_element(By.XPATH, '//button[text()="Price"]').click()

# Select the drop down menu and input $$20000
time.sleep(7)
maxprice = driver.find_element(By.CSS_SELECTOR, "input[placeholder='$100000']")
maxprice.clear()
maxprice.send_keys('20000')

# Click the Search button
time.sleep(10) 
driver.find_element(By.XPATH, "//button[contains(text(), 'Search')]").click()

# Select the location filter
time.sleep(6)
driver.find_element(By.XPATH, "//button[.//div[contains(text(), 'location')]]").click()

# Input zip code 94102 (UC Law Campus)
time.sleep(8)  # Wait for the location input to become visible
locationinput = driver.find_element(By.CSS_SELECTOR, "input[name='location']")
locationinput.clear()
locationinput.send_keys('94102')

# Select the address that pops up so it can autofill
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, "li[data-location='San Francisco, CA 94102, USA']").click()

# Select the radius filter
time.sleep(7)
driver.find_element(By.XPATH, "//button[.//div[contains(text(),'radius')]]").click()

# Select 25 miles from dropdown error
time.sleep(8)
driver.find_element(By.XPATH, "//li[text()='25 miles']").click()

# Loop through each page and save it as an html file locally
for i in range(50):
    time.sleep(5)

    # write results page to an html file locally
    with open(f'autolist_usedcars_94102_under20000/page{i+1}.html', 'w', encoding='utf-8') as f:
        f.write(driver.page_source)
    print(f'page {i} saved!')

    # Click on the next page
    time.sleep(8)
    driver.find_element(By.CSS_SELECTOR, "button[aria-label='next page']").click()

driver.quit()

page 0 saved!
page 1 saved!
page 2 saved!
page 3 saved!
page 4 saved!
page 5 saved!
page 6 saved!
page 7 saved!
page 8 saved!
page 9 saved!
page 10 saved!
page 11 saved!
page 12 saved!
page 13 saved!
page 14 saved!
page 15 saved!
page 16 saved!
page 17 saved!
page 18 saved!
page 19 saved!
page 20 saved!
page 21 saved!
page 22 saved!
page 23 saved!
page 24 saved!
page 25 saved!
page 26 saved!
page 27 saved!
page 28 saved!
page 29 saved!
page 30 saved!
page 31 saved!
page 32 saved!
page 33 saved!
page 34 saved!
page 35 saved!
page 36 saved!
page 37 saved!
page 38 saved!
page 39 saved!
page 40 saved!
page 41 saved!
page 42 saved!
page 43 saved!
page 44 saved!
page 45 saved!
page 46 saved!
page 47 saved!
page 48 saved!
page 49 saved!


## Extract Data For Each Car on Each Search Result Page

In [88]:
# Initialize the WebDriver outside of the loop
path = "/Users/avantikagoyal/Desktop/DDR/chromedriver-mac-arm64/chromedriver"
driver = webdriver.Chrome(executable_path=path)

''' FUNCTION TO EXTRACT EXTRA CAR DETAILS FROM CAR URL '''
def get_car_details(url):
    # Fetch the page
    driver.get(url)
    time.sleep(5)  # Consider using more robust wait conditions here

    # Get the HTML content
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # Select all car info in soup
    info = soup.select('div[class*="info-data"]')
    # Initialize dictionary to store extra car details
    car_details = {}
    
    # Extract car trim
    try:
        car_details['trim'] = info[0].text
    except IndexError:
        car_details['trim'] = 'None'

    # Extract car transmission type
    try:
        car_details['transmission'] = info[1].text
    except IndexError:
        car_details['transmission'] = 'None'

    # Extract car engine type
    try:
        car_details['engine'] = info[2].text
    except IndexError:
        car_details['engine'] = 'None'

    # Extract car drive type
    try:
        car_details['drivetrain'] = info[3].text
    except IndexError:
        car_details['drivetrain'] = 'None'

    # Extract car exterior color
    try:
        car_details['ext_color'] = info[5].text
    except IndexError:
        car_details['ext_color'] = 'None'

    # Extract car interior color
    try:
        car_details['int_color'] = info[6].text
    except IndexError:
        car_details['int_color'] = 'None'

    # Extract car style
    try:
        car_details['style'] = info[9].text
    except IndexError:
        car_details['style'] = 'None'

    return car_details

In [82]:
# Initialize list which will contain dictionaries for each car's data
all_car_data = []

''' LOOP OVER EACH HTML FILE AND EXTRACT CAR DATA '''
for i in range(50):

    # Open the html file and create a beautifulsoup object
    with open(f'autolist_usedcars_94102_under20000/page{i+1}.html') as f:
        soup = bs(f, 'html.parser')

    # Select all cars on results page
    results = soup.find_all('div', attrs={'data-testid':'search-result'})

    for car in results:

        # Select car title
        title = car.select_one('div[class*="year-make-model"]').text.replace('\xa0', ' ')

        # Store all car data into dictionary
        car_data = {}

        # Extract car title
        try:
            car_data['title'] = title
        except:
            car_data['title'] = 'None'

        # Extract car year
        try:
            car_data['year'] = re.findall('^\d{4}', title)[0]
        except:
            car_data['year'] = 'None'

        # Extract car make
        try:
            car_data['make'] = title.split()[1]
        except:
            car_data['make'] = 'None'

        # Extract car model
        try:
            car_data['model'] = ' '.join(title.split()[2:])
        except:
            car_data['model'] = 'None'

        # Extract car mileage
        try:
            car_data['miles'] = int(car.select_one('div[class*="mileage"]').text.split()[0].replace(',', ''))
        except:
            car_data['miles'] = 'None'

        # Extract car location
        try:
            car_data['location'] = re.findall('Miles(.+)', car.select_one('div[class*="miles-location"]').text)[0]
        except:
            car_data['location'] = 'None'

        # Extract car's days of market
        try:
            car_data['days_on_market'] = int(re.findall('\d+', car.select_one('div[class*="time-on-market"]').text)[0])
        except:
            car_data['days_on_market'] = 'None'

        # Extract car price
        try:
            car_data['price'] = int(re.findall('\d+', car.select_one('div[class*="right"]').text.replace('$', '').replace(',', ''))[0])
        except:
            car_data['price'] = 'None'

        # Extract car's estimated monthly payment
        try:
            car_data['est_monthly_pay'] = int(re.findall('\d+', car.select_one('div[class*="monthly-payment-value"]').text)[0])
        except:
            car_data['est_monthly_pay'] = 'None'

        # Extract car's URL
        try:
            car_data['URL'] = car.find('a').get('href')
        except:
            car_data['URL'] = 'None'

        # Fetch additional details from the car's page
        car_details = get_car_details(car_data['URL'])
        # Join the two dictionaries
        car_data.update(car_details)
        
        print(car_data)
        print('\n')

        # Save all data to a list of dictionaries
        all_car_data.append(car_data)

driver.quit()  # Quit the driver after processing all cars

{'title': '2013 Subaru Outback 2.5i Premium', 'year': '2013', 'make': 'Subaru', 'model': 'Outback 2.5i Premium', 'miles': 102533, 'location': 'Pleasanton, CA', 'days_on_market': 1, 'price': 14998, 'est_monthly_pay': 216, 'URL': 'https://www.autolist.com/listings#city=San%20Francisco&latitude=37.7786871&limit=20&longitude=-122.4212424&page=26&price_max=20000&radius=25&state=CA&vin=4S4BRBCC7D1234857', 'trim': '2.5i Premium', 'transmission': 'manual', 'engine': '2.5L', 'drivetrain': 'AWD', 'ext_color': 'gasoline', 'int_color': 'Gray', 'style': '102533'}


{'title': '2013 Infiniti G37 Journey', 'year': '2013', 'make': 'Infiniti', 'model': 'G37 Journey', 'miles': 102545, 'location': 'Pleasanton, CA', 'days_on_market': 4, 'price': 15998, 'est_monthly_pay': 231, 'URL': 'https://www.autolist.com/listings#city=San%20Francisco&latitude=37.7786871&limit=20&longitude=-122.4212424&page=26&price_max=20000&radius=25&state=CA&vin=JN1CV6AP4DM713255', 'trim': 'Journey', 'transmission': 'automatic', 'eng

In [83]:
# Check length of collection (number of cars)
len(all_car_data)

1000

## Store All Data in MongoDB Database

In [84]:
# Connect to MongoDB
mo_c = MongoClient()
client = MongoClient('localhost', 27017)

In [85]:
# Create used_cars database and bay_area_used_cars collection
# Insert docs into collection
db = client['used_cars']
used_cars_coll = db['bay_area_used_cars']

for doc in all_car_data:
    used_cars_coll.insert_one(doc)

In [91]:
# Print first 20 docs in collection
for doc in db['bay_area_used_cars'].find({}).limit(20):
    print(doc)

{'_id': ObjectId('65fb234ae2bca42e10d2092f'), 'title': '2015 Ford Transit Connect XLT', 'year': '2015', 'make': 'Ford', 'model': 'Transit Connect XLT', 'miles': 42692, 'location': 'Colma, CA', 'days_on_market': 21, 'price': 18491, 'est_monthly_pay': 267, 'URL': 'https://www.autolist.com/listings#city=San%20Francisco&latitude=37.7786871&limit=20&longitude=-122.4212424&page=1&price_max=20000&radius=25&state=CA&vin=NM0GS9F71F1224332', 'trim': 'XLT', 'transmission': 'automatic', 'engine': 'Duratec 2.5L I4', 'drivetrain': 'FWD', 'ext_color': 'gasoline', 'int_color': 'Tectonic Silver Metallic', 'style': '42692'}
{'_id': ObjectId('65fb234ae2bca42e10d20930'), 'title': '2016 BMW 4 Series 428i', 'year': '2016', 'make': 'BMW', 'model': '4 Series 428i', 'miles': 42419, 'location': 'San Jose, CA', 'days_on_market': 14, 'price': 19998, 'est_monthly_pay': 288, 'URL': 'https://www.autolist.com/listings#city=San%20Francisco&latitude=37.7786871&limit=20&longitude=-122.4212424&page=1&price_max=20000&radi